# Пайплайн <a id='top'></a>

## Imports

In [ ]:
import yaml

import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from data_transformer import DataTransformer
from models import BoostingModel
from scorer import MoneyScorer

# PATHS

In [ ]:
DATA_PATH = 'data/all_england_v2.8.csv'
FEATURES_PATH = 'data/features.yaml'

## Load data

In [3]:
raw_train_data = pd.read_csv(DATA_PATH)

with open(FEATURES_PATH) as f:
    all_features_dict = yaml.safe_load(f)

In [4]:
dict_type = type(all_features_dict)

In [5]:
for key, item in all_features_dict.items():
    if type(item) == dict_type:
        print(f"'{key}':")
        for inner_key in item.keys():
            print(f"\t'{inner_key}'")
    else:
        print(f"'{key}'")

        

'base_features'
'scored_features'
'result_coef_features'
'names':
	'team_names'
	'country_names'
	'city_names'
	'manager_names'
'manager_features'
'squad_features':
	'common_features'
	'detail_features'
'city_features'
'money_features'
'coefficients':
	'double_chance_features'
	'total_coef_features'
	'handicap_features'
	'half_features'
	'odd_features'
	'correct_score_features'
	'time_match_features'
	'both_scored_features'


In [6]:
# profile = ProfileReport(raw_train_data, minimal=True)
# profile.to_file("output.html")

## Preprocess data

In [7]:
def base_data_preprocess(data):
    
    preprocessed_data = data.copy()
    
#     preprocessed_data = preprocessed_data.fillna(1.01)
    preprocessed_data['day'] = pd.to_datetime(preprocessed_data.date, format='%d.%m.%Y')
    preprocessed_data['day_of_week'] = preprocessed_data['day'].dt.day_name()
    preprocessed_data['year'] = preprocessed_data['day'].dt.year
    preprocessed_data.date = preprocessed_data.day.values.astype(np.int64) // 10 ** 9
    preprocessed_data.country = 'England'
    preprocessed_data = preprocessed_data.sort_values(by='date')
    preprocessed_data = preprocessed_data.drop(columns=['link'])
    drop_index = preprocessed_data[preprocessed_data.home_goalkeepers_average_age.isna()].index
    preprocessed_data = preprocessed_data.drop(index=drop_index)

    return preprocessed_data

In [8]:
train_data = base_data_preprocess(raw_train_data)

train_data.tail(5)

,date,time,day_of_week,season,country,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,home_handicap_0,away_handicap_0,home_handicap_minus_15,away_handicap_minus_15,home_handicap_minus_1,away_handicap_minus_1,home_handicap_plus_15,away_handicap_plus_15,home_handicap_plus_1,away_handicap_plus_1,home_first_half_handicap_0,away_first_half_handicap_0,home_first_half_handicap_minus_15,away_first_half_handicap_minus_15,home_first_half_handicap_minus_1,away_first_half_handicap_minus_1,home_first_half_handicap_plus_15,away_first_half_handicap_plus_15,home_first_half_handicap_plus_1,away_first_half_handicap_plus_1,home_second_half_handicap_0,away_second_half_handicap_0,home_second_half_handicap_minus_15,away_second_half_handicap_minus_15,home_second_half_handicap_minus_1,away_second_half_handicap_minus_1,home_second_half_handicap_plus_15,away_second_half_handicap_plus_15,home_second_half_handicap_plus_1,away_second_half_handicap_plus_1,home_first_half_win_rate,draw_first_half_rate,away_first_half_win_rate,home_second_half_win_rate,draw_second_half_rate,away_second_half_win_rate,home_double_chance_first_half_rate,away_double_chance_first_half_rate,no_draw_first_half_rate,home_double_chance_second_half_rate,away_double_chance_second_half_rate,no_draw_second_half_rate,total_first_half_over_05_rate,total_first_half_under_05_rate,total_first_half_over_1_rate,total_first_half_under_1_rate,total_first_half_over_15_rate,total_first_half_under_15_rate,total_first_half_over_2_rate,total_first_half_under_2_rate,total_second_half_over_05_rate,total_second_half_under_05_rate,total_second_half_over_1_rate,total_second_half_under_1_rate,total_second_half_over_15_rate,total_second_half_under_15_rate,total_second_half_over_2_rate,total_second_half_under_2_rate,both_team_to_score_first_half_yes,both_team_to_score_first_half_no,both_team_to_score_second_half_yes,both_team_to_score_second_half_no,odd,even,odd_first,odd_second,even_first,even_second,correct_score10,correct_score20,correct_score21,correct_score30,correct_score31,correct_score32,correct_score40,correct_score41,correct_score00,correct_score11,correct_score22,correct_score33,correct_score01,correct_score02,correct_score12,correct_score03,correct_score13,correct_score23,correct_score04,correct_score14,home_home,home_draw,home_away,draw_home,draw_draw,draw_away,away_home,away_draw,away_away,home_scored,away_scored,home_first_half_scored,away_first_half_scored,home_second_half_scored,away_second_half_scored,home_manager_working_days,away_manager_working_days,home_manager_name,away_manager_name,home_manager_start_date,home_manager_birthday,home_manager_country,away_manager_start_date,away_manager_birthday,away_manager_country,month,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,home_stadium,home_stadium_capacity,home_city,away_city,is_derby,home_goalkeepers_average_age,home_defenders_average_age,home_midfields_average_age,home_attacks_average_age,home_goalkeepers_total_market_value,home_defenders_total_market_value,home_midfields_total_market_value,home_attacks_total_market_value,home_goalkeepers_e_market_value,home_defenders_e_market_value,home_midfields_e_market_value,home_attacks_e_market_value,home_is_manager_and_league_same_country,away_goalkeepers_average_age,away_defenders_average_age,away_midfields_average_age,away_attacks_average_age,away_goalkeepers_total_market_value,away_defenders_total_market_value,away_midfields_total_market_value,away_attacks_total_market_value,away_goal

In [9]:
train_data.shape

(9655, 210)

In [10]:
nan_count = train_data.isna().sum().sort_values(ascending=False)
nan_count = nan_count[nan_count > 0]
nan_col_index = nan_count.index
nan_count

home_first_half_handicap_plus_15      160
home_first_half_handicap_plus_1       104
home_handicap_plus_15                  73
away_first_half_handicap_minus_15      42
home_second_half_handicap_plus_15      41
home_handicap_plus_1                   37
away_first_half_handicap_minus_1       27
home_second_half_handicap_plus_1       23
total_over_1_rate                      20
away_handicap_minus_15                 14
away_second_half_handicap_minus_15     13
away_handicap_minus_1                  13
away_second_half_handicap_minus_1       4
home_handicap_0                         2
dtype: int64

In [11]:
def prepare_for_yaml(features_list) -> str:
    
    result_yaml = ''
    
    for feature in features_list:
        result_yaml += f"- '{feature}'\n"
        
    return result_yaml

# Разбиение фичей на группы [ПРОПУСТИТЬ](#numeric)

In [ ]:
enumerated_features = dict()

for idx, feature in enumerate(list(train_data.columns)):
    enumerated_features[idx] = feature

In [ ]:
enumerated_features

In [ ]:
base_features = list(train_data.iloc[0:2, 0:9].columns)
base_features.append(enumerated_features[126])
base_features.append(enumerated_features[127])
base_features.append(enumerated_features[208])
base_features.append(enumerated_features[209])
base_features.append('month')


print(prepare_for_yaml(base_features))

In [ ]:
squad_features = list(train_data.iloc[0:2,143:153].columns)
print(prepare_for_yaml(squad_features))

In [ ]:
squad_detail_features = list(train_data.iloc[0:2, 158:208])
print(prepare_for_yaml(squad_detail_features[:10]))

In [ ]:
money_features = list(train_data.filter(like='market_value', axis=1).columns) + list(train_data.filter(like='s_sum', axis=1).columns)
print(prepare_for_yaml(money_features))

In [ ]:
city_features = list(train_data.iloc[0:2,153:158].columns)
print(prepare_for_yaml(city_features))

In [ ]:
result_coef_features = [enumerated_features[9], enumerated_features[10], enumerated_features[11]]
print(prepare_for_yaml(result_coef_features))

In [ ]:
double_chance_features = list(train_data.filter(like='chance', axis=1).columns)
print(prepare_for_yaml(double_chance_features))

In [ ]:
total_coef_features = list(train_data.filter(like='total_over', axis=1).columns) + list(train_data.filter(like='total_under', axis=1).columns)
print(prepare_for_yaml(total_coef_features))

In [ ]:
handicap_features = list(train_data.iloc[0:2,29:39].columns)
print(prepare_for_yaml(handicap_features))

In [ ]:
half_features = list(train_data.iloc[0:2,39:91].columns)
print(prepare_for_yaml(half_features[:10]))

In [ ]:
odd_features = list(train_data.iloc[0:2,91:97].columns)

print(prepare_for_yaml(odd_features))

In [ ]:
correct_score_features = list(train_data.filter(like='correct_score', axis=1).columns)
print(prepare_for_yaml(correct_score_features))

In [ ]:
time_match_features = list(train_data.iloc[0:2,117:126].columns)
print(prepare_for_yaml(time_match_features))

In [ ]:
both_scored_features = list(train_data.iloc[0:2,87:91].columns)
both_scored_features += list(train_data.iloc[0:2,128:132].columns)
both_scored_features.append(enumerated_features[27])
both_scored_features.append(enumerated_features[28])

print(prepare_for_yaml(both_scored_features))

In [ ]:
manager_features = tuple(train_data.filter(like='manager', axis=1).columns)

print(prepare_for_yaml(manager_features))

<a id='numeric'></a>


<a id='transformer'></a>
# Трансформер 

[наверх](#top)

In [12]:
numeric_features = tuple(train_data.select_dtypes(include=['int', 'float']).columns)
print(prepare_for_yaml(numeric_features[:10]))
print(f'Number of numeric features: {len(numeric_features)}')

- 'date'
- 'league_level'
- 'home_win_rate'
- 'draw_rate'
- 'away_win_rate'
- 'home_double_chance_rate'
- 'away_double_chance_rate'
- 'no_draw_rate'
- 'total_over_1_rate'
- 'total_under_1_rate'

Number of numeric features: 195


In [13]:
categorical_features = tuple(train_data.select_dtypes(include=['object']).columns)
print(prepare_for_yaml(categorical_features))
print(f'Number of numeric features: {len(categorical_features)}')

- 'time'
- 'day_of_week'
- 'season'
- 'country'
- 'league'
- 'home_team'
- 'away_team'
- 'home_manager_name'
- 'away_manager_name'
- 'home_manager_country'
- 'away_manager_country'
- 'home_stadium'
- 'home_city'
- 'away_city'

Number of numeric features: 14


In [156]:
features = {'cat_features':categorical_features,
            'num_features':numeric_features,
            'grouped_features':all_features_dict
           }

transformer_context = {'data':train_data,
                       'features':features
                      }

transformer = DataTransformer(transformer_context)

In [157]:
train, val, test, decode_labels, encode_labels = transformer.run_logic() 

All categorical features are already encoded!
Features are already generated!


In [158]:
test.head()

,date,time,day_of_week,season,country,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,home_handicap_0,away_handicap_0,home_handicap_minus_15,away_handicap_minus_15,home_handicap_minus_1,away_handicap_minus_1,home_handicap_plus_15,away_handicap_plus_15,home_handicap_plus_1,away_handicap_plus_1,home_first_half_handicap_0,away_first_half_handicap_0,home_first_half_handicap_minus_15,away_first_half_handicap_minus_15,home_first_half_handicap_minus_1,away_first_half_handicap_minus_1,home_first_half_handicap_plus_15,away_first_half_handicap_plus_15,home_first_half_handicap_plus_1,away_first_half_handicap_plus_1,home_second_half_handicap_0,away_second_half_handicap_0,home_second_half_handicap_minus_15,away_second_half_handicap_minus_15,home_second_half_handicap_minus_1,away_second_half_handicap_minus_1,home_second_half_handicap_plus_15,away_second_half_handicap_plus_15,home_second_half_handicap_plus_1,away_second_half_handicap_plus_1,home_first_half_win_rate,draw_first_half_rate,away_first_half_win_rate,home_second_half_win_rate,draw_second_half_rate,away_second_half_win_rate,home_double_chance_first_half_rate,away_double_chance_first_half_rate,no_draw_first_half_rate,home_double_chance_second_half_rate,away_double_chance_second_half_rate,no_draw_second_half_rate,total_first_half_over_05_rate,total_first_half_under_05_rate,total_first_half_over_1_rate,total_first_half_under_1_rate,total_first_half_over_15_rate,total_first_half_under_15_rate,total_first_half_over_2_rate,total_first_half_under_2_rate,total_second_half_over_05_rate,total_second_half_under_05_rate,total_second_half_over_1_rate,total_second_half_under_1_rate,total_second_half_over_15_rate,total_second_half_under_15_rate,total_second_half_over_2_rate,total_second_half_under_2_rate,both_team_to_score_first_half_yes,both_team_to_score_first_half_no,both_team_to_score_second_half_yes,both_team_to_score_second_half_no,odd,even,odd_first,odd_second,even_first,even_second,correct_score10,correct_score20,correct_score21,correct_score30,correct_score31,correct_score32,correct_score40,correct_score41,correct_score00,correct_score11,correct_score22,correct_score33,correct_score01,correct_score02,correct_score12,correct_score03,correct_score13,correct_score23,correct_score04,correct_score14,home_home,home_draw,home_away,draw_home,draw_draw,draw_away,away_home,away_draw,away_away,home_manager_working_days,away_manager_working_days,home_manager_name,away_manager_name,home_manager_start_date,home_manager_birthday,home_manager_country,away_manager_start_date,away_manager_birthday,away_manager_country,month,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,home_stadium,home_stadium_capacity,home_city,away_city,is_derby,home_goalkeepers_average_age,home_defenders_average_age,home_midfields_average_age,home_attacks_average_age,home_goalkeepers_total_market_value,home_defenders_total_market_value,home_midfields_total_market_value,home_attacks_total_market_value,home_goalkeepers_e_market_value,home_defenders_e_market_value,home_midfields_e_market_value,home_attacks_e_market_value,home_is_manager_and_league_same_country,away_goalkeepers_average_age,away_defenders_average_age,away_midfields_average_age,away_attacks_average_age,away_goalkeepers_total_market_value,away_defenders_total_market_value,away_midfields_total_market_value,away_attacks_total_market_value,away_goalkeepers_e_market_value,away_defenders_e_market_value,away_midfields_e_market_value,away_attacks_e_market_value,away_is

In [17]:
train.target.value_counts()

3    4135
0    2947
1    2503
Name: target, dtype: int64

In [18]:
print(f"Class_0 rate {train.target.value_counts()[0] / train.shape[0]}")
print(f"Class_1 rate {train.target.value_counts()[1] / train.shape[0]}")
print(f"Class_3 rate {train.target.value_counts()[3] / train.shape[0]}")

Class_0 rate 0.30745957224830467
Class_1 rate 0.26113719353155973
Class_3 rate 0.43140323422013566


# Проверить данные на:

- мультиколлинеарность
- forward/backward selection
- 


# Переход к [модели](#model)

## Generating some features and testing hypotheses

# Оставим только "не-коэффициенты" и декодируем названия команд 

In [ ]:
def _set_target(row):
    """ Set target feature from score """

    if row.home_scored > row.away_scored:
        return 3
    elif row.home_scored == row.away_scored:
        return 1
    else:
        return 0

train_data_base = train_data[all_features_dict['base_features'] + all_features_dict['squad_features']].copy()
train_data_base['target'] = train_data_base.apply(_set_target, axis=1)
train_data_base.tail()

In [ ]:
for feature in money_features:
    train_data_base[f"log_{feature}"] = train_data_base[feature].apply(np.log)

train_data_base.sample(5)

# ПЕРЕДЕЛАТЬ! Считаем сезонные total :
- забитые
- пропущенные
- количество очков

In [ ]:
query = '((home_team == @team) | (away_team == @team)) & (season == @season) & (league == @league)'

season_data = pd.DataFrame()

total_features = train_data_base.copy()

for league in total_features.league.unique():

    for season in total_features.season.unique():
        
        season_league_data = total_features.query('(season == @season) & (league == @league)')
        
        for team in season_league_data.home_team.unique():   
            
            season_data = season_league_data.query('((home_team == @team) | (away_team == @team))')
            
            total_points = 0
            total_scored = 0
            total_missed = 0


            for idx in season_data.index:

                if season_data.loc[idx, 'home_team'] == team:

                    total_points += season_data.loc[idx, 'target']
                    total_scored += season_data.loc[idx, 'home_scored']
                    total_missed += season_data.loc[idx, 'away_scored']

                else:

                    home = season_data.loc[idx, 'home_scored']
                    away = season_data.loc[idx, 'away_scored']

                    away_match_score = 3 if home < away else 1 if home == away else 0

                    total_points += away_match_score
                    total_scored += season_data.loc[idx, 'away_scored']
                    total_missed += season_data.loc[idx, 'home_scored']

            condition_home = ((total_features.home_team == team) & (total_features.season == season))
            condition_away = ((total_features.away_team == team) & (total_features.season == season))
                    
            total_features.loc[condition_home,'total_points_home'] = total_points
            total_features.loc[condition_away,'total_points_away'] = total_points
            
            total_features.loc[condition_home,'total_scored_home'] = total_scored
            total_features.loc[condition_away,'total_scored_away'] = total_scored

            total_features.loc[condition_home,'total_missed_home'] = total_missed
            total_features.loc[condition_away,'total_missed_away'] = total_missed

            total_features.loc[condition_home,'total_diff_home'] = total_scored - total_missed
            total_features.loc[condition_away,'total_diff_away'] = total_scored - total_missed            



            print(f"TEAM: {team} LEAGUE: {league} SEASON: {season}\ntotal points:{total_points}\ttotal scored:{total_scored}\ttotal missed:{total_missed}\ttotal_diff: {total_scored - total_missed}")
            print('_________')
        
total_features.tail()        

In [ ]:
home_team_alltime = train_data_base.groupby(['home_team'], as_index=False) \
                        .agg(home_mean_scored_season = ('home_scored', 'mean'),
                            home_mean_missed_season = ('away_scored', 'mean')                                 
                            )

away_team_alltime = train_data_base.groupby(['away_team'], as_index=False) \
                        .agg(away_mean_scored_alltime = ('home_scored', 'mean'),
                            away_mean_missed_alltime = ('away_scored', 'mean')                                 
                            )

home_team_alltime.head()

In [ ]:
train_data_base = train_data_base.merge(home_team_alltime, how='left', on=['home_team', 'season'])
train_data_base = train_data_base.merge(away_team_alltime, how='left', on=['away_team', 'season'])

train_data_base.head()

# ПЕРЕДЕЛАТЬ! Личные встречи за все время забитые/пропущенные

In [ ]:
# personal_battles = train_with_names.groupby(['home_team', 'away_team'], as_index=False) \
#                         .agg(total_home_scored = ('home_scored', 'sum'),
#                             total_home_missed = ('away_scored', 'sum'),
                            
#                             )

# personal_battles

# Подсчет текущих очков и вин/луз стриков по сезонам

In [ ]:
query = '((home_team == @team) | (away_team == @team)) & (league == @season)'

def calculate_win_streak(actual_win_streak: int, match_result: int) -> int:
    
    new_win_streak = actual_win_streak
    
    if match_result == 3:
        
        new_win_streak += 1
    
    else:
        
        new_win_streak = 0
        
    return new_win_streak

def calculate_lose_streak(actual_lose_streak: int, match_result: int) -> int:
    
    new_lose_streak = actual_lose_streak
    
    if match_result == 0:
        
        new_lose_streak += 1
    
    else:
        
        new_lose_streak = 0
        
    return new_lose_streak


train_with_current_points = train_data_base.copy()

for season in train_with_current_points.season.unique():
    
    for team in train_with_current_points.home_team.unique():    

        current_points = 0
        current_win_streak = 0
        current_lose_streak = 0

        team_season_data = train_with_current_points.query(query)

        for idx in team_season_data.index:
            
#             if season == 'premier-league-2021-2022' and team == 'Arsenal':

#                 print(f"Match of {team_season_data.loc[idx, 'date']}\n"
#                          f"Home team: {team_season_data.loc[idx, 'home_team']}, "
#                          f"away team: {team_season_data.loc[idx, 'away_team']}"
#                          )

            if team_season_data.loc[idx, 'home_team'] == team:

                train_with_current_points.loc[idx, 'home_current_points'] = current_points

                current_points += team_season_data.loc[idx, 'target']

#                 if season == 'premier-league-2021-2022' and team == 'Arsenal':
    
#                     print(f"result is {team_season_data.loc[idx, 'target']} points for home team {team_season_data.loc[idx, 'home_team']}\n"
#                          f"Match score {team_season_data.loc[idx, 'home_scored']}:{team_season_data.loc[idx, 'away_scored']}")

                train_with_current_points.loc[idx, 'home_current_lose_streak'] = current_lose_streak

                train_with_current_points.loc[idx, 'home_current_win_streak'] = current_win_streak

                current_lose_streak = calculate_lose_streak(current_lose_streak, team_season_data.loc[idx, 'target'])

                current_win_streak = calculate_win_streak(current_win_streak, team_season_data.loc[idx, 'target'])

            else:

                train_with_current_points.loc[idx, 'away_current_points'] = current_points

                home = team_season_data.loc[idx, 'home_scored']
                away = team_season_data.loc[idx, 'away_scored']

                away_match_score = 3 if home < away else 1 if home == away else 0

                current_points += away_match_score
                
#                 if season == 'premier-league-2021-2022' and team == 'Arsenal':

#                     print(f"result is {away_match_score} points for away team {team_season_data.loc[idx, 'away_team']}\n"
#                          f"Match score {home}:{away}")

                train_with_current_points.loc[idx, 'away_current_lose_streak'] = current_lose_streak

                train_with_current_points.loc[idx, 'away_current_win_streak'] = current_win_streak

                current_lose_streak = calculate_lose_streak(current_lose_streak, away_match_score)

                current_win_streak = calculate_win_streak(current_win_streak, away_match_score)
            
#             if season == 'premier-league-2021-2022' and team == 'Arsenal':

#                 print('______________________________________________________________ \n')

train_with_current_points.home_current_points = train_with_current_points.home_current_points.astype(int)
train_with_current_points.away_current_points = train_with_current_points.away_current_points.astype(int)
train_with_current_points.away_current_win_streak = train_with_current_points.away_current_win_streak.astype(int)
train_with_current_points.away_current_lose_streak = train_with_current_points.away_current_lose_streak.astype(int)
train_with_current_points.home_current_win_streak = train_with_current_points.home_current_win_streak.astype(int)
train_with_current_points.home_current_lose_streak = train_with_current_points.home_current_lose_streak.astype(int)

In [ ]:
train_with_current_points[(train_with_current_points.home_team == 'Manchester City')|(train_with_current_points.away_team == 'Manchester City')].tail(15)

In [ ]:
train_with_names = train_with_names.merge(train_with_current_points, how='left')
train_with_names.sample(5)

# <a id='model'></a> Model  
[go to transformer](#transformer)

[go to top](#top)

In [161]:
train = train.reset_index()
train = train.drop(columns=['index'])
train.head()

,date,time,day_of_week,season,country,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,home_handicap_0,away_handicap_0,home_handicap_minus_15,away_handicap_minus_15,home_handicap_minus_1,away_handicap_minus_1,home_handicap_plus_15,away_handicap_plus_15,home_handicap_plus_1,away_handicap_plus_1,home_first_half_handicap_0,away_first_half_handicap_0,home_first_half_handicap_minus_15,away_first_half_handicap_minus_15,home_first_half_handicap_minus_1,away_first_half_handicap_minus_1,home_first_half_handicap_plus_15,away_first_half_handicap_plus_15,home_first_half_handicap_plus_1,away_first_half_handicap_plus_1,home_second_half_handicap_0,away_second_half_handicap_0,home_second_half_handicap_minus_15,away_second_half_handicap_minus_15,home_second_half_handicap_minus_1,away_second_half_handicap_minus_1,home_second_half_handicap_plus_15,away_second_half_handicap_plus_15,home_second_half_handicap_plus_1,away_second_half_handicap_plus_1,home_first_half_win_rate,draw_first_half_rate,away_first_half_win_rate,home_second_half_win_rate,draw_second_half_rate,away_second_half_win_rate,home_double_chance_first_half_rate,away_double_chance_first_half_rate,no_draw_first_half_rate,home_double_chance_second_half_rate,away_double_chance_second_half_rate,no_draw_second_half_rate,total_first_half_over_05_rate,total_first_half_under_05_rate,total_first_half_over_1_rate,total_first_half_under_1_rate,total_first_half_over_15_rate,total_first_half_under_15_rate,total_first_half_over_2_rate,total_first_half_under_2_rate,total_second_half_over_05_rate,total_second_half_under_05_rate,total_second_half_over_1_rate,total_second_half_under_1_rate,total_second_half_over_15_rate,total_second_half_under_15_rate,total_second_half_over_2_rate,total_second_half_under_2_rate,both_team_to_score_first_half_yes,both_team_to_score_first_half_no,both_team_to_score_second_half_yes,both_team_to_score_second_half_no,odd,even,odd_first,odd_second,even_first,even_second,correct_score10,correct_score20,correct_score21,correct_score30,correct_score31,correct_score32,correct_score40,correct_score41,correct_score00,correct_score11,correct_score22,correct_score33,correct_score01,correct_score02,correct_score12,correct_score03,correct_score13,correct_score23,correct_score04,correct_score14,home_home,home_draw,home_away,draw_home,draw_draw,draw_away,away_home,away_draw,away_away,home_manager_working_days,away_manager_working_days,home_manager_name,away_manager_name,home_manager_start_date,home_manager_birthday,home_manager_country,away_manager_start_date,away_manager_birthday,away_manager_country,month,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,home_stadium,home_stadium_capacity,home_city,away_city,is_derby,home_goalkeepers_average_age,home_defenders_average_age,home_midfields_average_age,home_attacks_average_age,home_goalkeepers_total_market_value,home_defenders_total_market_value,home_midfields_total_market_value,home_attacks_total_market_value,home_goalkeepers_e_market_value,home_defenders_e_market_value,home_midfields_e_market_value,home_attacks_e_market_value,home_is_manager_and_league_same_country,away_goalkeepers_average_age,away_defenders_average_age,away_midfields_average_age,away_attacks_average_age,away_goalkeepers_total_market_value,away_defenders_total_market_value,away_midfields_total_market_value,away_attacks_total_market_value,away_goalkeepers_e_market_value,away_defenders_e_market_value,away_midfields_e_market_value,away_attacks_e_market_value,away_is

In [99]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score
from catboost import CatBoostClassifier, Pool

In [21]:
cat_features = list(categorical_features)

model_data = {
    'train':train,
    'val':val,
    'test':test,
    'target':'target',
    'cat_features':cat_features
}

In [22]:
X_test = test.drop(columns=['target'])
y_test = test.target

In [152]:
def test_cv_generator(data, base_size=50, window=30):
    for i in range(0, len(data) - base_size - window + 1, window):
        train = data[:base_size+i]
        val = data[base_size+i:base_size+i+window]
        yield train, val

In [153]:
test_data = [i for i in range(150)]
gen = test_cv_generator(test_data)
gen

<generator object test_cv_generator at 0x7fb8fa686740>

In [154]:
i = 0
for train, val in gen:
    print(f"CV [{i}]")
    print(f"Train: {train}")
    print(f"Val: {val}")
    print('_____')
    i += 1

CV [0]
Train: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Val: [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
_____
CV [1]
Train: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
Val: [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
_____
CV [2]
Train: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,

In [162]:
def get_cv_data(train_data, train_initial_size=7000, window=50):
    
    for index in range(0, train_data.shape[0] - train_initial_size - window + 1, window):
        
        train_cv = train_data.loc[:train_initial_size + i]
        val_cv = train_data.loc[train_initial_size + i: train_initial_size + i + window]
        yield train_cv, val_cv

In [164]:
cv_generator = get_cv_data(train)
train_1, val_1 = next(cv_generator)
val_1

,date,time,day_of_week,season,country,league,league_level,home_team,away_team,home_win_rate,draw_rate,away_win_rate,home_double_chance_rate,away_double_chance_rate,no_draw_rate,total_over_1_rate,total_under_1_rate,total_over_15_rate,total_under_15_rate,total_over_2_rate,total_under_2_rate,total_over_25_rate,total_under_25_rate,total_over_3_rate,total_under_3_rate,total_over_35_rate,total_under_35_rate,both_team_to_score_yes,both_team_to_score_no,home_handicap_0,away_handicap_0,home_handicap_minus_15,away_handicap_minus_15,home_handicap_minus_1,away_handicap_minus_1,home_handicap_plus_15,away_handicap_plus_15,home_handicap_plus_1,away_handicap_plus_1,home_first_half_handicap_0,away_first_half_handicap_0,home_first_half_handicap_minus_15,away_first_half_handicap_minus_15,home_first_half_handicap_minus_1,away_first_half_handicap_minus_1,home_first_half_handicap_plus_15,away_first_half_handicap_plus_15,home_first_half_handicap_plus_1,away_first_half_handicap_plus_1,home_second_half_handicap_0,away_second_half_handicap_0,home_second_half_handicap_minus_15,away_second_half_handicap_minus_15,home_second_half_handicap_minus_1,away_second_half_handicap_minus_1,home_second_half_handicap_plus_15,away_second_half_handicap_plus_15,home_second_half_handicap_plus_1,away_second_half_handicap_plus_1,home_first_half_win_rate,draw_first_half_rate,away_first_half_win_rate,home_second_half_win_rate,draw_second_half_rate,away_second_half_win_rate,home_double_chance_first_half_rate,away_double_chance_first_half_rate,no_draw_first_half_rate,home_double_chance_second_half_rate,away_double_chance_second_half_rate,no_draw_second_half_rate,total_first_half_over_05_rate,total_first_half_under_05_rate,total_first_half_over_1_rate,total_first_half_under_1_rate,total_first_half_over_15_rate,total_first_half_under_15_rate,total_first_half_over_2_rate,total_first_half_under_2_rate,total_second_half_over_05_rate,total_second_half_under_05_rate,total_second_half_over_1_rate,total_second_half_under_1_rate,total_second_half_over_15_rate,total_second_half_under_15_rate,total_second_half_over_2_rate,total_second_half_under_2_rate,both_team_to_score_first_half_yes,both_team_to_score_first_half_no,both_team_to_score_second_half_yes,both_team_to_score_second_half_no,odd,even,odd_first,odd_second,even_first,even_second,correct_score10,correct_score20,correct_score21,correct_score30,correct_score31,correct_score32,correct_score40,correct_score41,correct_score00,correct_score11,correct_score22,correct_score33,correct_score01,correct_score02,correct_score12,correct_score03,correct_score13,correct_score23,correct_score04,correct_score14,home_home,home_draw,home_away,draw_home,draw_draw,draw_away,away_home,away_draw,away_away,home_manager_working_days,away_manager_working_days,home_manager_name,away_manager_name,home_manager_start_date,home_manager_birthday,home_manager_country,away_manager_start_date,away_manager_birthday,away_manager_country,month,home_squad_size,home_average_age,home_amount_of_foreigners,home_e_market_value,home_total_market_value,away_squad_size,away_average_age,away_amount_of_foreigners,away_e_market_value,away_total_market_value,home_stadium,home_stadium_capacity,home_city,away_city,is_derby,home_goalkeepers_average_age,home_defenders_average_age,home_midfields_average_age,home_attacks_average_age,home_goalkeepers_total_market_value,home_defenders_total_market_value,home_midfields_total_market_value,home_attacks_total_market_value,home_goalkeepers_e_market_value,home_defenders_e_market_value,home_midfields_e_market_value,home_attacks_e_market_value,home_is_manager_and_league_same_country,away_goalkeepers_average_age,away_defenders_average_age,away_midfields_average_age,away_attacks_average_age,away_goalkeepers_total_market_value,away_defenders_total_market_value,away_midfields_total_market_value,away_attacks_total_market_value,away_goalkeepers_e_market_value,away_defenders_e_market_value,away_midfields_e_market_value,away_attacks_e_market_value,away_is

In [ ]:
model_params = {'n_estimators':1000,
                'learning_rate':0.01,
                'loss_function':'MultiClass',
                'task_type':'GPU',
                'verbose':250
                }

model = CatBoostClassifier(**model_params)

In [23]:
# model_params = {'n_estimators':1000,
#                 'learning_rate':0.01,
#                 'loss_function':'MultiClass',
#                 'task_type':'GPU',
#                 'verbose':250
#                 }

# model = BoostingModel(params=model_params, data=model_data)

# model.fit()
# preds_class = model.predict(X_test)
# preds_proba = model.predict_proba(X_test)

0:	learn: 1.0970040	test: 1.0979961	best: 1.0979961 (0)	total: 17.3ms	remaining: 17.3s
250:	learn: 0.9990590	test: 1.0795494	best: 1.0722525 (86)	total: 3.83s	remaining: 11.4s
500:	learn: 0.9761152	test: 1.0892192	best: 1.0722525 (86)	total: 7.87s	remaining: 7.83s
750:	learn: 0.9545336	test: 1.0903403	best: 1.0722525 (86)	total: 11.6s	remaining: 3.85s
999:	learn: 0.9351588	test: 1.0947804	best: 1.0722525 (86)	total: 15.4s	remaining: 0us
bestTest = 1.072252502
bestIteration = 86
Shrink model to first 87 iterations.


# Оценка качества

In [24]:
print(f'Validation ACCURACY SCORE: {accuracy_score(y_test, preds_class)}')

Validation ACCURACY SCORE: 0.55


In [25]:
print(f'Validation ROC AUC SCORE: {roc_auc_score(y_test, preds_proba, multi_class="ovr")}')

Validation ROC AUC SCORE: 0.4869378306878307


In [26]:
train_preds_class = model.model.predict(train.drop(columns=['target']))

In [27]:
train_preds_df = pd.DataFrame()
train_preds_df['pred_class'] = train_preds_class.ravel()

In [28]:
print('True target rate: ')
print(f"Class_0 rate {train.target.value_counts()[0] / train.shape[0]}")
print(f"Class_1 rate {train.target.value_counts()[1] / train.shape[0]}")
print(f"Class_3 rate {train.target.value_counts()[3] / train.shape[0]}")

True target rate: 
Class_0 rate 0.30745957224830467
Class_1 rate 0.26113719353155973
Class_3 rate 0.43140323422013566


In [30]:
train_preds_df.pred_class.value_counts()

3    6675
0    2910
Name: pred_class, dtype: int64

In [31]:
print('Predict target rate:')
print(f"Class_0 rate {train_preds_df.pred_class.value_counts()[0] / train_preds_df.shape[0]}")
# print(f"Class_1 rate {train_preds_df.pred_class.value_counts()[1] / train_preds_df.shape[0]}")
print(f"Class_3 rate {train_preds_df.pred_class.value_counts()[3] / train_preds_df.shape[0]}")

Predict target rate:
Class_0 rate 0.30359937402190923
Class_3 rate 0.6964006259780907


In [32]:
results = pd.DataFrame()

results['true'] = y_test
results['pred_class'] = preds_class
results['score_0'] = preds_proba[:,0]
results['score_1'] = preds_proba[:,1]
results['score_3'] = preds_proba[:,2]

results

,true,pred_class,score_0,score_1,score_3
6891,3,0,0.398686,0.302882,0.298433
9333,3,3,0.228844,0.258639,0.512517
6766,0,0,0.432595,0.287196,0.280209
9510,3,3,0.298932,0.310747,0.390320
9408,0,0,0.399739,0.291138,0.309124
7018,0,3,0.217480,0.237801,0.544719
6964,3,3,0.303694,0.312797,0.383510
6837,0,3,0.343315,0.312490,0.344195
9604,1,3,0.276102,0.307684,0.416214
9669,1,0,0.488301,0.263340,0.248359


# Влияние фичей на качество модели

In [38]:
feature_importance = model.get_feature_importances()
feature_importance_df = pd.DataFrame()

feature_importance_df['feature'] = X_test.columns
feature_importance_df['importance'] = feature_importance

good_features = feature_importance_df[np.abs(feature_importance_df.importance) > 0].sort_values(by='importance',ascending=False).head(30)
bad_features = feature_importance_df.sort_values(by='importance',ascending=True).head(30)

print(f'Number of features {X_test.shape[1]}')

Number of features 240


In [39]:
good_features

,feature,importance
9,home_win_rate,7.668776
10,draw_rate,6.231026
11,away_win_rate,5.006873
64,away_second_half_win_rate,4.338611
14,no_draw_rate,4.218730
36,away_handicap_plus_15,3.831842
62,home_second_half_win_rate,3.671404
39,home_first_half_handicap_0,3.444516
29,home_handicap_0,2.902997
13,away_double_chance_rate,2.707952


In [40]:
bad_features

,feature,importance
119,home_away,0.0
169,away_goalkeepers_total_market_value,0.0
166,away_defenders_average_age,0.0
164,home_is_manager_and_league_same_country,0.0
162,home_midfields_e_market_value,0.0
158,home_midfields_total_market_value,0.0
151,is_derby,0.0
150,away_city,0.0
149,home_city,0.0
147,home_stadium,0.0


In [42]:
test_predictions = pd.DataFrame()
test_predictions['home_team'] = X_test.home_team.map(decode_labels['team_names'])
test_predictions['away_team'] = X_test.away_team.map(decode_labels['team_names'])
test_predictions['home_win_proba'] = preds_proba[:,0]
test_predictions['draw_proba'] = preds_proba[:,1]
test_predictions['away_win_proba'] = preds_proba[:,2]
test_predictions['home_win_rate'] = X_test.home_win_rate
test_predictions['draw_rate'] = X_test.draw_rate
test_predictions['away_win_rate'] = X_test.away_win_rate
test_predictions['result'] = y_test
test_predictions['predict'] = preds_class

In [43]:
test_predictions

,home_team,away_team,home_win_proba,draw_proba,away_win_proba,home_win_rate,draw_rate,away_win_rate,result,predict
6891,Crewe Alexandra,AFC Wimbledon,0.398686,0.302882,0.298433,3.51,3.28,2.18,3,0
9333,Exeter City,Colchester United,0.228844,0.258639,0.512517,1.50,4.12,6.49,3,3
6766,Fleetwood Town,Oxford United,0.432595,0.287196,0.280209,4.38,3.77,1.79,0,0
9510,Bristol Rovers,Salford City,0.298932,0.310747,0.390320,2.33,3.23,3.15,3,3
9408,Oldham Athletic,Northampton Town,0.399739,0.291138,0.309124,3.46,3.17,2.21,0,0
7018,Wigan Athletic,Cambridge United,0.217480,0.237801,0.544719,1.33,5.05,9.31,0,3
6964,Rotherham United,Ipswich Town,0.303694,0.312797,0.383510,2.22,3.25,3.43,3,3
6837,Milton Keynes Dons,Sheffield Wednesday,0.343315,0.312490,0.344195,2.63,3.20,2.80,0,3
9604,Salford City,Barrow AFC,0.276102,0.307684,0.416214,1.92,3.30,4.29,1,3
9669,Scunthorpe United,Stevenage FC,0.488301,0.263340,0.248359,7.60,4.32,1.44,1,0


# Считаем ROI - "коэффициент выгодности ставки"

In [44]:
class ROIChecker():
    def __init__(self, predictions):
        self.predictions = predictions
    
    def get_roi(self):
        
        self.predictions['home_win_ROI'] = self.predictions.home_win_rate * self.predictions.home_win_proba - 1
        self.predictions['away_win_ROI'] = self.predictions.away_win_rate * self.predictions.away_win_proba - 1
        self.predictions['draw_ROI'] = self.predictions.draw_rate * self.predictions.draw_proba - 1
        
        return self.predictions

In [45]:
checker = ROIChecker(test_predictions)
roi_info = checker.get_roi()

In [46]:
roi_info.head()

,home_team,away_team,home_win_proba,draw_proba,away_win_proba,home_win_rate,draw_rate,away_win_rate,result,predict,home_win_ROI,away_win_ROI,draw_ROI
6891,Crewe Alexandra,AFC Wimbledon,0.398686,0.302882,0.298433,3.51,3.28,2.18,3,0,0.399387,-0.349417,-0.006548
9333,Exeter City,Colchester United,0.228844,0.258639,0.512517,1.50,4.12,6.49,3,3,-0.656734,2.326237,0.065591
6766,Fleetwood Town,Oxford United,0.432595,0.287196,0.280209,4.38,3.77,1.79,0,0,0.894766,-0.498426,0.082729
9510,Bristol Rovers,Salford City,0.298932,0.310747,0.390320,2.33,3.23,3.15,3,3,-0.303487,0.229508,0.003714
9408,Oldham Athletic,Northampton Town,0.399739,0.291138,0.309124,3.46,3.17,2.21,0,0,0.383096,-0.316837,-0.077094


In [103]:
def explain_roi_info(roi_info):
    roi_cols = ['home_win_ROI', 'away_win_ROI', 'draw_ROI']
    best_roi_df = pd.DataFrame()
    for index, row in roi_info.iterrows():
        print(f"Match #{index+1}: {row.home_team} vs {row.away_team}")
        
        max_roi = np.max(roi_info.loc[index, roi_cols])
        current_choice = 'home_win_ROI'
        
        for col in roi_cols:
            if row[col] == max_roi:
                current_choice = col
        current_choice = ' '.join(current_choice.split('_')[:1])
        
        if max_roi > 0:
            print(f"Maximal ROI = {np.round(max_roi*100,2)}% on {current_choice}")
        
        else:
            print("Нет положительного ROI")
            
        print('_______________________________ \n')
        
        best_roi_df.loc[index, 'home_team'] = row.home_team
        best_roi_df.loc[index, 'away_team'] = row.away_team
        best_roi_df.loc[index, 'best_ROI'] = max_roi
        best_roi_df.loc[index, 'choice'] = current_choice
        best_roi_df.loc[index, 'home_win_rate'] = row.home_win_rate
        best_roi_df.loc[index, 'draw_rate'] = row.draw_rate
        best_roi_df.loc[index, 'away_win_rate'] = row.away_win_rate
        best_roi_df.loc[index, 'result'] = row.result
        best_roi_df.loc[index, 'predict'] = row.predict

        
    return best_roi_df

In [104]:
best_roi_df = explain_roi_info(roi_info)

Match #6892: Crewe Alexandra vs AFC Wimbledon
Maximal ROI = 39.94% on home
_______________________________ 

Match #9334: Exeter City vs Colchester United
Maximal ROI = 232.62% on away
_______________________________ 

Match #6767: Fleetwood Town vs Oxford United
Maximal ROI = 89.48% on home
_______________________________ 

Match #9511: Bristol Rovers vs Salford City
Maximal ROI = 22.95% on away
_______________________________ 

Match #9409: Oldham Athletic vs Northampton Town
Maximal ROI = 38.31% on home
_______________________________ 

Match #7019: Wigan Athletic vs Cambridge United
Maximal ROI = 407.13% on away
_______________________________ 

Match #6965: Rotherham United vs Ipswich Town
Maximal ROI = 31.54% on away
_______________________________ 

Match #6838: Milton Keynes Dons vs Sheffield Wednesday
Нет положительного ROI
_______________________________ 

Match #9605: Salford City vs Barrow AFC
Maximal ROI = 78.56% on away
_______________________________ 

Match #9670: Scunt

In [105]:
best_roi_df

,home_team,away_team,best_ROI,choice,home_win_rate,draw_rate,away_win_rate,result,predict
6891,Crewe Alexandra,AFC Wimbledon,0.399387,home,3.51,3.28,2.18,3.0,0.0
9333,Exeter City,Colchester United,2.326237,away,1.50,4.12,6.49,3.0,3.0
6766,Fleetwood Town,Oxford United,0.894766,home,4.38,3.77,1.79,0.0,0.0
9510,Bristol Rovers,Salford City,0.229508,away,2.33,3.23,3.15,3.0,3.0
9408,Oldham Athletic,Northampton Town,0.383096,home,3.46,3.17,2.21,0.0,0.0
7018,Wigan Athletic,Cambridge United,4.071338,away,1.33,5.05,9.31,0.0,3.0
6964,Rotherham United,Ipswich Town,0.315438,away,2.22,3.25,3.43,3.0,3.0
6837,Milton Keynes Dons,Sheffield Wednesday,-0.000032,draw,2.63,3.20,2.80,0.0,3.0
9604,Salford City,Barrow AFC,0.785557,away,1.92,3.30,4.29,1.0,3.0
9669,Scunthorpe United,Stevenage FC,2.711086,home,7.60,4.32,1.44,1.0,0.0


In [193]:
def money_score(best_roi_df, bet=100):
    results = {'home':3, 'draw':1, 'away':0}
    cols = {'home':'home_win_rate', 'draw':'draw_rate', 'away':'away_win_rate'}
    profit = 0
    skipped_bets = 0
    accepted_bets = 0
    total_coef = 0
    win_coef = 0
    lose_coef = 0
    win_bets = 0
    lose_bets = 0
    total_bank = 0
    win_bank = 0
    lose_bank = 0
    
    for index, row in best_roi_df.iterrows():
        if row.best_ROI < 1:
                skipped_bets += 1
        else:
            accepted_bets += 1

            if results[row.choice] == row.result:

                current_profit = bet * (row[cols[row.choice]] - 1)
                profit += current_profit
                win_coef += row[cols[row.choice]]

                print(f'Match {row.home_team} vs {row.away_team}')
                print(row.choice, row[cols[row.choice]], current_profit)
                print(f'Match score ')
                print('_____________________________________________________\n')
                win_bank += current_profit
                win_bets += 1
                
            else:
                profit -= bet
                lose_bank -= bet
                lose_bets += 1
                lose_coef += row[cols[row.choice]]
                
            total_coef += row[cols[row.choice]]
            total_bank += bet

    average_win_coef = win_coef / win_bets

    average_lose_coef = lose_coef / lose_bets
    average_coef = total_coef / accepted_bets
    percent_profit = profit / total_bank * 100
          
    result = ((skipped_bets, accepted_bets),
            (profit, percent_profit),
            (total_bank, win_bank, lose_bank),
            (win_bets, lose_bets),
            (average_coef, average_win_coef, average_lose_coef))
    
    return result
    

In [194]:
score = money_score(best_roi_df, 100)

Match Wigan Athletic vs Cambridge United
away 9.31 831.0
Match score 
_____________________________________________________

Match Carlisle United vs Mansfield Town
home 4.7 370.0
Match score 
_____________________________________________________



In [195]:
print(f'Skipped bets: {score[0][0]}\t Accepted bets: {score[0][1]}')
print(f"Win bets: {score[3][0]}\t Lose bets: {score[3][1]}")
print(f"Win bank: {score[2][1]}$\t Lose bank: {score[2][2]}$")

print(f"Average coefficient: {np.round(score[4][0],2)}")
print(f"Avg win coef: {np.round(score[4][1],2)}\t Avg lose coef: {np.round(score[4][2],2)}")
print(f"Initial bank was {score[2][0]}$ \t Finish bank is {score[2][0]+np.round(score[1][0],2)}$\n")
print('___________RESULT__________')
print(f" \tWe get + {np.round(score[1][1],2)}% profit")
print(f" \tProfit: {score[1][0]}$")

Skipped bets: 14	 Accepted bets: 6
Win bets: 2	 Lose bets: 4
Win bank: 1201.0$	 Lose bank: -400$
Average coefficient: 6.96
Avg win coef: 7.01	 Avg lose coef: 6.94
Initial bank was 600$ 	 Finish bank is 1401.0$

___________RESULT__________
 	We get + 133.5% profit
 	Profit: 801.0$


# [НАВЕРХ](#top)